In [2]:
import numpy as np
import pandas as pd

# Classifier algorithms
from sklearn.ensemble import RandomForestClassifier

from flask import Flask, jsonify, request

import joblib
import json
    
def pre_processing(data):  
 
    #fill null values
    data['Gender'] = data["Gender"].fillna(data['Gender'].mode()[0])
    data['Married'] = data["Married"].fillna(data['Married'].mode()[0])
    data['Self_Employed'] = data["Self_Employed"].fillna(data['Self_Employed'].mode()[0])
    data['Dependents'] = data["Dependents"].fillna(data['Dependents'].mode()[0])
    data['Credit_History'] = data["Credit_History"].fillna(data['Credit_History'].mode()[0])
    data['LoanAmount'] = data["LoanAmount"].fillna(data['LoanAmount'].median()).astype(int)
    data['Loan_Amount_Term'] = data["Loan_Amount_Term"].fillna(data['Loan_Amount_Term'].mode()[0]).astype(int)

    #rename the variabeles with binary values
    data['is_male'] = np.where(data["Gender"]=='Male',1,0)
    data['is_Married'] = np.where(data["Married"]=='Yes',1,0)
    data['is_Self_Employed'] = np.where(data["Self_Employed"]=='Yes',1,0)
    data['Credit_History_availability'] = np.where(data["Credit_History"]==1.0,1,0)
    data['is_graduate'] = np.where(data["Education"]=='Graduate',1,0)
    
    labels = ['<90', '90-180', '180-360', '>360']
    bin_edges = [0, 90, 180, 360, np.inf]

    data['Loan_Amount_Term_grp'] = pd.cut(x=data['Loan_Amount_Term'], bins=bin_edges, labels=labels)
    data['Loan_Amount_Term_grp'].value_counts()
    
    #get dummies
    data = data.join(pd.get_dummies(data['Loan_Amount_Term_grp'], prefix='Loan_Term'))
    data = data.join(pd.get_dummies(data['Property_Area'], prefix='Property_Area'))
    data = data.join(pd.get_dummies(data['Dependents'], prefix='Dependents'))

    # Select Columns
    X_variables = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'is_Married',
                   'Credit_History_availability', 'is_graduate','Loan_Term_180-360','Loan_Term_>360', 'Property_Area_Rural',
                   'Property_Area_Semiurban', 'Dependents_2']
        
    # Assign 0 to missing columns
    for x in list(set(X_variables) - set(data.columns)):
        data[x] = 0

    return data[X_variables]

def post_processing(prediction):
    if len(prediction)==1:
        return prediction[:, 1][0]
    else:
        return prediction[:, 1]

def app_prediction_function(input_data, model):
    return post_processing(model.predict_proba(pre_processing(input_data), model=model))

app = Flask(__name__)

# Load model
model_file = 'model_rf.pickle'
model = joblib.load(model_file)
print(model)

@app.route("/")
def index():
    return "Greetings from Prediction API"

@app.route("/classifier", methods=['POST'])
def classifier():
    if request.method == 'POST': 
        input_data =  request.form.to_dict()
        print(input_data)
        print(type(input_data))
        input_data = pd.DataFrame([input_data])
        print(input_data)
        prediction = app_prediction_function(input_data, model)
        return jsonify({'prediction': prediction})  
        
if __name__ == '__main__':
    app.run(debug=True, port=5001)

RandomForestClassifier(max_depth=10, n_estimators=500, n_jobs=3, verbose=1)
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

C:\Users\rwidlk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
